In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
#%matplotlib notebook

In [ ]:
df_snowt = pd.read_csv(r'C:\Users\GGbri\OneDrive\Practicum Code\snowfall_telem.csv')
df_snowt.replace(['City', 'Clearwater', 'Ells', 'Frostbite', 'Mckay', 'North', 'McLellan','GRapids','SAth', 'Steepbank'],
                        [1,2,3,4,5,6,7,8,9,10],inplace=True)

new_df_snowt = df_snowt.dropna()
new_df_snowt.head(30)


In [ ]:
new_df_snowt.info()

In [ ]:
# import plotly.express as px
# center= dict(lat=new_df_snowt["Latitude"].median(),lon=new_df_snowt["Longitude"].median())
# fig=px.scatter_mapbox(new_df_snowt,lat="Latitude",lon="Longitude",animation_frame="Day",color="Pack",center=center,height=1000)
# fig.update_layout(mapbox_style="open-street-map",title_text="Plotly Animation of Packs per Day")
# fig.show()

In [ ]:
new_df_snowt.set_index('Day', inplace=True)
new_df_snowt.sort_index(inplace=True)
new_df_snowt

In [ ]:
dfm16=new_df_snowt.loc[(new_df_snowt['Month']) & (new_df_snowt['Pack'] == 6)]
dfm17=new_df_snowt.loc[(new_df_snowt['Month']) & (new_df_snowt['Pack'] == 7)]
df_month1_pack67 = pd.concat([dfm16, dfm17])
df_month1_pack67.info()


In [ ]:
df_month1_pack67.index

In [ ]:
fig = plt.figure(1, figsize=(15, 15))
for i,gp in df_month1_pack67.groupby("Day"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude','Pack']].values
    plt.subplot(8,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("Latitude, Longitude of Pack 6 & 7 per Day")
plt.tight_layout()    
plt.show()

#     actual=df_month1_pack67['Pack'].value_counts()
#     actual =np.array(actual)
#actual=np.bincount(df_month1_pack67['Pack'])
#print(actual)

# the label to predict is the id of the pack
#y = df_month1_pack67['Pack']
# y values are 6,7 Each corresponds to the following target names. 
#print(np.unique(y))
#print(y)

## From GUDHI documentation: ToMATO algorithm uses a density estimator and a neighborhood graph, starts with a mode-seeking phase (naive hill-climbing) to build initial clusters, and finishes by merging clusters based on their prominence.

I am choosing the number of clusters based on how many packs the algorithm is running.
The barcode diagram shows prominence as their distance to the diagonal. "There is always one point infinitely far: there is at least one cluster. Among the others, one point seems significantly farther from the diagonal than the others, which indicates that splitting the points into however many clusters" (Gudhi Software package).

In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_month1_pack67.groupby("Day"): 
    #display(i)
    #display(gp.head(10))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=100,n_clusters=2)
    t.fit(data)
    # Use subplots to display diagram and barcode side by side
    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 6 and 7 per Day") 
    t.plot_diagram()
    print(t.diagram_)
    #d_array=t.diagram_

plt.tight_layout()
plt.show()

In [ ]:
##Here I am subtracting x-y

# Note that the index starts from day 10-29 
#day 23 [[5.01911259 4.25176379]]
#[23,0.7673488]
#day 24  [[5.04370431 4.91608458]]
#[24,0.12761973]
#day 25  [[5.1634498  5.03023749]]
#[25,0.13321231]

In [ ]:
d67= {'Day': [23,24,25], 
     'BarcodeDistance': [0.7673488,0.12761973,0.13321231]}
df_67 = pd.DataFrame(data=d67)
df_67

line1,=plt.plot(df_67["Day"],df_67["BarcodeDistance"],label='packs 6 and 7 barcode length')
plt.legend(handles=[line1])
plt.title('Time Series of Packs 6 and 7 Day and Barcode Distance from diagonal \n')
plt.xlabel("Day"); plt.ylabel("Barcode distance")

In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_month1_pack67.groupby("Day"): 
    #display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=100,n_clusters=2)
    t.fit(data)
    #t.plot_diagram()
    labels = [6 if i ==0 else 7 for i in t.labels_]
    plt.subplot(8,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 
plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 6 and 7 per Day")
plt.tight_layout()    
plt.show()



In [ ]:
from plotly.subplots import make_subplots
#fig = plt.figure(1, figsize=(10, 10)) 
#fig = make_subplots(rows=8, cols=4)
#fig.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 6 and 7 per Day")
import gudhi
from gudhi.clustering.tomato import Tomato
import numpy as np

for (i,gp) in df_month1_pack67.groupby("Day"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude', 'Pack']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=100,n_clusters=2)
    t.fit(data)
    labels = [6 if i ==0 else 7 for i in t.labels_]
    import plotly.express as px
    center= dict(lat=X["Latitude"].median(),lon=X["Longitude"].median())
    fig=px.scatter_mapbox(data,lat=0,lon=1,animation_frame=X.index,color=X.Pack,color_continuous_scale=px.colors.sequential.Bluered,
                          center=center,width=500,height=400,size_max=20)
    fig.update_layout(mapbox_style='open-street-map',
                      title_text="ToMATo Plotly Subplots of Packs 6 and 7 per Day")
    fig.show()
   # mapbox_style="open-street-map"

In [ ]:
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# #Get the confusion matrix
# cf_matrix = confusion_matrix(y, t_bincount)
# print(cf_matrix)
# group_counts = ["{0:0.0f}".format(value) for value in
#                 cf_matrix.flatten()]

# group_percentages = ["{0:.2%}".format(value) for value in
#                      cf_matrix.flatten()/np.sum(cf_matrix)]

# labels = [f"{v1}\n{v2}\n" for v1, v2 in
#           zip(group_counts,group_percentages)]

# labels = np.asarray(labels).reshape(4,4)
# ax=sns.heatmap(cf_matrix, square=True, annot=labels, fmt='', cbar=True)
# ax.set_title('Pack 6 and 7 per Day ToMATo Clustering versus Actual Confusion Matrix \n');
# ax.set_xlabel('\nPredicted Pack Category')
# ax.set_ylabel('Actual Pack Category ')
# #ax.xaxis.set_ticklabels(['Pack 6','Pack 7'])
# #ax.yaxis.set_ticklabels(['pack 6','Pack 7'])
# plt.show()

## Packs 9,2,1

In [ ]:
dfm9=new_df_snowt.loc[(new_df_snowt['Pack'] == 9)]
dfm2=new_df_snowt.loc[(new_df_snowt['Pack'] == 2)]
dfm1=new_df_snowt.loc[(new_df_snowt['Pack'] == 1)]
dfm10=new_df_snowt.loc[(new_df_snowt['Pack'] == 10)]
df_pack92110= pd.concat([dfm9, dfm2,dfm1,dfm10])
df_pack92110.info()

In [ ]:
df_pack92110.index

In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 

for i,gp in df_pack92110.groupby("Day"): 
    #display(i)
    #display(gp.head(2))
    X=gp[['Latitude', 'Longitude','Pack']].values
    plt.subplot(8,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude')

plt.suptitle("Latitude, Longitude of Packs 9,2,1 and 10 per Day")
plt.tight_layout()  
plt.show()

#     actual = df_pack921["Pack"].value_counts()
#     print(actual)

# # the label to predict is the id of the person
# y = df_pack921['Pack']
# # y values corresponds to the following target names. 
# print(np.unique(y))
# print(y)

In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack92110.groupby("Day"): 
    # display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=85, n_clusters=4)
    t.fit(data)
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 9,2, 1 and 10 per Day") 
    t.plot_diagram()
    print(t.diagram_)

#take the matrices
plt.tight_layout()
plt.show()
    

In [ ]:
# # day 10 [[4.89662773 4.55453302]]

# [10,0.34209471]

# # day 12   [[4.28114887 4.24133937]]


# [12,0.0398095]
# # day 13  [[6.41641233 5.86838517]]

# [13,0.54802716]
# # day 17 [[3.36282565 3.11162412]]

# [17,0.25120153]
# # day 18  [[4.35978067 3.82779062]]

# [18,0.53199005]
# # day 21  [[4.75358162 4.65137434]
# #          [4.77983524 4.41887234]]

# [21,0.10220728]
# [21,0.3609629]

# # day 22  [[5.32377823 4.77869507]]

# [22,0.54508316]

# # day 23 [[5.30508955 5.27044727]
# #         [6.2444856  4.46495897]]

# [23,0.03464228]
# [23,1.77952663]

# # day 24 [[5.29032956 4.70393091]]

# [24,0.58639865]

In [ ]:
d92110 = {'Day': [10, 12,13,17,18,21,21,22,23,23,24], 
     'BarcodeDistance': [0.34209471,0.0398095,0.54802716,0.25120153,0.53199005,0.10220728,0.3609629,0.54508316
                        ,0.03464228,1.77952663,0.58639865]}
df_92110 = pd.DataFrame(data=d92110)
df_92110

line1,=plt.plot(df_92110["Day"],df_92110["BarcodeDistance"],label='packs 9,2,1 and 10 barcode length')
plt.legend(handles=[line1])
plt.title('Time Series of Packs 9,2,1,and 10 Day and Barcode Distance from diagonal \n')
plt.xlabel("Day"); plt.ylabel("Barcode distance")

In [ ]:
fig = plt.figure(1, figsize=(15, 15))
import gudhi
from gudhi.clustering.tomato import Tomato

# fig = plt.figure(1, figsize=(10, 10)) 
for (i,gp) in df_pack92110.groupby("Day"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']].values
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X
    t = Tomato(graph_type='knn',density_type='logDTM', k=85, n_clusters=4)
    t.fit(data)
    #t.plot_diagram()
    labels = [9 if i ==0 else 2 if i==1 else 1 if i==2 else 10 for i in t.labels_]
    plt.subplot(8,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 
plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Packs 9, 2, 1 and 10 per Day")
plt.tight_layout()    
plt.show()


# tcount=np.bincount(t.labels_)
# print(tcount)
# t_bincount=(t.labels_)
# print(t_bincount)

# print("Number of tomato clusters for pack 9 is ",tcount[0])
# print("Number of tomato clusters for pack 2 is ", tcount[1])
# print("Number of tomato clusters for pack 1 is ", tcount[2])
# #print("Number of tomato clusters for pack undecided is ", tcount[3])

In [ ]:
from plotly.subplots import make_subplots
#fig = plt.figure(1, figsize=(10, 10)) 
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack92110.groupby("Day"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude', 'Pack']]
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=85,n_clusters=4)
    t.fit(data)
    labels = [9 if i ==0 else 2 if i==1 else 1 if i==2 else 10 for i in t.labels_]
    import plotly.express as px
    center= dict(lat=X["Latitude"].median(),lon=X["Longitude"].median())
    fig=px.scatter_mapbox(data,lat=0,lon=1,animation_frame=X.index,color=X.Pack,color_continuous_scale=px.colors.sequential.Bluered,
                          center=center,width=650, height=450)
    fig.update_layout(mapbox_style="open-street-map",title_text="ToMATo Plotly Subplots of Packs 9, 2 1, and 10 per Day")
    fig.show()

## Packs 5,4,3

In [ ]:
dfm5=new_df_snowt.loc[(new_df_snowt['Pack'] == 5)]
dfm4=new_df_snowt.loc[(new_df_snowt['Pack'] == 4)]
dfm3=new_df_snowt.loc[(new_df_snowt['Pack'] == 3)]
df_pack543= pd.concat([dfm5, dfm4,dfm3])
df_pack543.info()

In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 

for i,gp in df_pack543.groupby("Day"): 
#     display(i)
#     display(gp.head(2))

    X=gp[['Latitude', 'Longitude','Pack']].values
    plt.subplot(8,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    plt.xlabel('Latitude'); plt.ylabel('Longitude')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
plt.suptitle("Latitude, Longitude of Packs 5,4 & 3 per Day")
plt.tight_layout()
plt.show()
# actual = df_pack543["Pack"].value_counts()
# # the label to predict is the id of the pack
# y = df_pack543['Pack']
# # y values are 5,4,3 Each corresponds to the following target names. 
# print(np.unique(y))
# print(y)

In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack543.groupby("Day"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=100,n_clusters=3)
    t.fit(data)
    # Use subplots to display diagram and barcode side by side
    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    #gudhi.plot_persistence_barcode(persistence=data,legend=True,axes=axes[0])
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 5,4 and 3 per Day") 
    t.plot_diagram()
    print(t.diagram_)

plt.tight_layout()
plt.show()
    

In [ ]:
# # day 15 [[4.70987289 4.62785974]]

# [15,0.08201315]

# # day 17 [[5.82880327 5.23923133]]

# [17,0.58957194]

# # day 18 [[4.56588987 4.5043014 ]
# #          [5.53852993 4.4483529 ]]
# [18,0.06158847]
# [18,1.09017703]

# # day 19 [[5.49552585 5.33090979]]
# [19,0.16461606]

# # day 20 [[5.45577224 5.3291497 ]]

# [20,0.12662254]
# # day 21 [[5.08245277 4.91258052]]
# [21,0.16987225]

In [ ]:
d543= {'Day': [15,17,18,18,19,20,21], 
     'BarcodeDistance': [0.08201315,0.58957194,0.06158847,1.09017703,
                         0.16461606,0.12662254,0.16987225]}
df_543 = pd.DataFrame(data=d543)
df_543

line1,=plt.plot(df_543["Day"],df_543["BarcodeDistance"],label='packs 5,4,3 barcode length')
plt.legend(handles=[line1])
plt.title('Time Series of Packs 5,4, and 3 Day and Barcode Distance from diagonal \n')
plt.xlabel("Day"); plt.ylabel("Barcode distance")

In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 
#fig.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 6 and 7 per Day")
import gudhi
from gudhi.clustering.tomato import Tomato

# fig = plt.figure(1, figsize=(10, 10)) 
for (i,gp) in df_pack543.groupby("Day"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']].values
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X
    t = Tomato(graph_type='knn',density_type='logDTM', k=100,n_clusters=3)
    t.fit(data)
    #t.plot_diagram()
    labels = [5 if i ==0 else 4 if i==1 else 3 for i in t.labels_]
    plt.subplot(8,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Packs 5, 4 and 3 per Day")   
plt.tight_layout()    
plt.show()

# tcount=np.bincount(t.labels_)
# print(tcount)
# t_bincount=(t.labels_)
# print(t_bincount)

# print("Number of tomato clusters for pack 5 is ",tcount[0])
# print("Number of tomato clusters for pack 4 is ", tcount[1])
# print("Number of tomato clusters for pack 3 is ", tcount[2])
#print("Number of tomato clusters for pack undecided is ", tcount[3])

In [ ]:
from plotly.subplots import make_subplots
fig = plt.figure(1, figsize=(15, 15)) 
fig = make_subplots(rows=8, cols=4)
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack543.groupby("Day"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude', 'Pack']]
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X.values
    t = Tomato(density_type='logDTM', k=100, n_clusters=3)
    t.fit(data)
    labels = [5 if i ==0 else 4 if i==1 else 3 for i in t.labels_]
    #plt.subplot(8,4,i)
    #t.plot_diagram()
    import plotly.express as px
    center= dict(lat=X["Latitude"].median(),lon=X["Longitude"].median())
    fig=px.scatter_mapbox(data,lat=0,lon=1,animation_frame=X.index,color=X.Pack,color_continuous_scale=px.colors.sequential.Bluered,
                          center=center,width=750,height=440)
    fig.update_layout(mapbox_style="open-street-map",title_text="ToMATo Plotly Subplots of Packs 5, 4 and 3 per Day")
    fig.show()
    
#plt.suptitle("ToMATo Persistence-based diagram of Latitude/Longitude of Pack 5, 4 and 3 per Day") 
plt.tight_layout()
plt.show()

## Snowfall Category Clustering: with aim to use the timeline of snowfall to understand if packs are constricting (meaning their points are tigher or vice versa, expanding).

In [ ]:
df_month1_pack67.replace(['two_before', 'one_before', 'day_of_snowfall', 'one_after', 'two_after', 'three_after', 'control'],                      
                         [1,2,3,4,5,6,7],inplace=True)
df_month1_pack67

df_pack92110.replace(['two_before', 'one_before', 'day_of_snowfall', 'one_after', 'two_after', 'three_after', 'control'],                      
                         [1,2,3,4,5,6,7],inplace=True)
df_pack92110

df_pack543.replace(['two_before', 'one_before', 'day_of_snowfall', 'one_after', 'two_after', 'three_after', 'control'],                      
                         [1,2,3,4,5,6,7],inplace=True)
df_pack543

In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 
for i,gp in df_month1_pack67.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude','Pack']].values
    plt.subplot(8,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("Latitude, Longitude of Pack 6 & 7 given Snowfall Category")
plt.tight_layout()    
plt.show()

In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_month1_pack67.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=105,n_clusters=2)
    t.fit(data)
    # Use subplots to display diagram and barcode side by side
    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
#     gudhi.plot_persistence_barcode(persistence=t.diagram_,legend=True,axes=axes[0])
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 6 and 7 given Snowfall Category") 
    t.plot_diagram()
    print(t.diagram_)
    

plt.tight_layout()
plt.show()

In [ ]:
# # #['two_before', 'one_before', 'day_of_snowfall', 'one_after', 'two_after', 'three_after', 'control'],                      
# #                         # [1,2,3,4,5,6,7])
    
# # two_before  [[5.60256749 4.4833868 ]]

# distance barcode is 1.11918069

# # one_before [[6.21874992 5.43879757]]

# distance barcode is 0.77995235

# # day_of_snowfall [[5.84040536 5.05506953]]

# distance barcode is 0.78533583

# # one_after [[6.10110416 5.52310327]]
# distance barcode is 0.57800089

# # two_after [[6.38681012 6.26442767]
# #              [5.71895798 5.55702058]]

# distance barcode 1 is 0.12238245
# distance barcode 2 is 0.1619374

# # three_after [[6.08936831 5.85518589]
# #              [6.13516566 5.59235166]]

# distance barcode 1 is 0.23418242
# distance barcode 2 is 0.542814
# # control [[5.97458404 5.45874528]]


In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_month1_pack67.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=105,n_clusters=2)
    t.fit(data)
    #t.plot_diagram()
    labels = [6 if i ==0 else 7 for i in t.labels_]
    plt.subplot(8,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 
plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 6 and 7 given Snowfall Category")
plt.tight_layout()    
plt.show()



In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 
for i,gp in df_pack92110.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude','Pack']].values
    plt.subplot(8,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)

    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("Latitude, Longitude of Pack 9,2,1 and 10 given Snowfall Category")
plt.tight_layout()    
plt.show()

In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack92110.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=140,n_clusters=4)
    t.fit(data)
    # Use subplots to display diagram and barcode side by side
    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
#     gudhi.plot_persistence_barcode(persistence=t.diagram_,legend=True,axes=axes[0])
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 9,2,1, and 10 given Snowfall Category") 
    t.plot_diagram()
    print(t.diagram_)
    

plt.tight_layout()
plt.show()

In [ ]:
## Time Series of snowfall category

# # #['two_before', 'one_before', 'day_of_snowfall', 'one_after', 'two_after', 'three_after', 'control'],                      
# # #                         # [1,2,3,4,5,6,7])
    
# # # two_before  [[4.16232864 3.54399949]
# #                  [3.77196096 3.54399949]]

# distance barcode 1 is 0.61832915
# distance barcode 2 is 0.22796147

# # # one_before [[5.35432332 4.17740296]]
# distance barcode is 1.17692036

# # # day_of_snowfall [[5.42006425 5.05988787]
# #                      [5.19212327 4.8708456 ]]

# distance barcode 1 is 0.36017638
# distance barcode 2 is 0.32127767

# # # one_after [[7.15855411 4.48216451]]
# distance barcode is 2.6763896

# # # two_after [[7.30641694 5.82425768]]
# # #             

# distance barcode is 1.48215926

# # # three_after [[5.99666182 5.52966576]
# #                  [5.02218462 4.92895637]]
# # #              
# distance barcode 1 is 0.46699606
# distance barcode 2 is 0.09322825
# # # control [[7.56362759 5.88390014]]


In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack92110.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=140,n_clusters=4)
    t.fit(data)
    #t.plot_diagram()
    labels = [9 if i ==0 else 2 if i==1 else 1 if i==2 else 10 for i in t.labels_]
    plt.subplot(8,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 
plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 9,2,1 and 10 given Snowfall Category")
plt.tight_layout()    
plt.show()


In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 
for i,gp in df_pack543.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude','Pack']].values
    plt.subplot(8,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("Latitude, Longitude of Pack 5,4, and 3 given Snowfall Category")
plt.tight_layout()    
plt.show()

In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack543.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=110,n_clusters=3)
    t.fit(data)
    # Use subplots to display diagram and barcode side by side
    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
#     gudhi.plot_persistence_barcode(persistence=t.diagram_,legend=True,axes=axes[0])
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 5,4, and 3 given Snowfall Category") 
    t.plot_diagram()
    print(t.diagram_)
    

plt.tight_layout()
plt.show()

In [ ]:
# # #['two_before', 'one_before', 'day_of_snowfall', 'one_after', 'two_after', 'three_after', 'control'],                      
# # # #                         # [1,2,3,4,5,6,7])
    
# # # # two_before  
# # #                  [[6.08143647 5.43412473]
# #                      [6.59739291 4.89423674]
# #                      [6.65733224 4.53107116]]

# distance barcode 1 is 0.64731174
# distance barcode 2 is 1.70315617
# distance barcode 3 is 2.12626108


# # # # one_before 

# # [[6.11788806 5.89832439]
# #  [5.95457632 5.24306661]
# #  [8.3578043  5.07016385]]


# distance barcode 1 is 0.21956367
# distance barcode 2 is 0.71150971
# distance barcode 3 is 3.28764045

# # # # day_of_snowfall 
# # #                     

# # [[7.56141897 5.99822575]
# #  [6.68196174 5.62592226]
# #  [7.77233616 5.57749627]]


# distance barcode 1 is 1.56319322
# distance barcode 2 is 1.05603948
# distance barcode 3 is 2.19483989


# # # # one_after 

# # [[8.07917517 5.84866136]
# #  [6.6347803  5.04825226]]

# distance barcode 1 is 2.23051381
# distance barcode 2 is 1.58652804


# # # # two_after 
# # # #     

# # [[6.92805872 6.76877193]
# #  [7.11361929 6.06659849]
# #  [5.85635142 5.3402036 ]
# #  [5.42698729 5.20535412]]


# distance barcode 1 is 0.15928679
# distance barcode 2 is 1.0470208
# distance barcode 3 is 0.51614782
# distance barcode 4 is 0.22163317

# # # # three_after 
# # #                 
# # # #      [[6.74453716 6.03746851]
# #          [7.45581697 5.99843177]]   

# distance barcode 1 is 0.70706865
# distance barcode 2 is 1.4573852


# # # # control 



In [ ]:
fig = plt.figure(1, figsize=(15, 15)) 
#fig.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 6 and 7 per Day")
import gudhi
from gudhi.clustering.tomato import Tomato

# fig = plt.figure(1, figsize=(10, 10)) 
for (i,gp) in df_pack543.groupby("snowfall_category"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']].values
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X
    t = Tomato(graph_type='knn',density_type='logDTM', k=110,n_clusters=3)
    t.fit(data)
    #t.plot_diagram()
    labels = [5 if i ==0 else 4 if i==1 else 3 for i in t.labels_]
    plt.subplot(8,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Packs 5, 4 and 3 given Snowfall Category")   
plt.tight_layout()    
plt.show()

# tcount=np.bincount(t.labels_)
# print(tcount)
# t_bincount=(t.labels_)
# print(t_bincount)

# print("Number of tomato clusters for pack 5 is ",tcount[0])
# print("Number of tomato clusters for pack 4 is ", tcount[1])
# print("Number of tomato clusters for pack 3 is ", tcount[2])
#print("Number of tomato clusters for pack undecided is ", tcount[3])

In [ ]:
print(df_month1_pack67['SnowDepth'])

In [ ]:
#print(df_month1_pack67["SnowDepth"])
fig = plt.figure(1, figsize=(15, 25)) 
for i,gp in df_month1_pack67.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude','Longitude','Pack']].values
    plt.subplot(25,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("Latitude, Longitude of Pack 6 & 7 given SnowDepth")
plt.tight_layout()  
plt.show()

# the label to predict is the id of the pack
#y = df_month1_pack67['Pack']
# y values are 6,7 Each corresponds to the following target names. 
#print(np.unique(y))
#print(y)

#actual=df_month1_pack67['SnowDepth'].value_counts()
#     actual =np.array(actual)
#actual=np.bincount(df_month1_pack67['SnowDepth'])
#print(actual)


In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_month1_pack67.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=100,n_clusters=2)
    t.fit(data)
    # Use subplots to display diagram and barcode side by side
    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    #gudhi.plot_persistence_barcode(persistence=data,legend=True,axes=axes[0])
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 6 and 7 given SnowDepth") 
    t.plot_diagram()
    print(t.diagram_)

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(1, figsize=(15, 25)) 
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_month1_pack67.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=100,n_clusters=2)
    t.fit(data)
    #t.plot_diagram()
    labels = [6 if i ==0 else 7 for i in t.labels_]
    plt.subplot(25,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 
    #y=np.bincount(labels)
    #print(y)
   
plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 6 and 7 given SnowDepth")
plt.tight_layout()    
plt.show()


# the label to predict is the id of the pack
#y = df_month1_pack67['Pack']
# y values are 6,7 Each corresponds to the following target names. 
#print(np.unique(y))
#print(y)

In [ ]:
fig = plt.figure(1, figsize=(15, 25)) 
for i,gp in df_pack92110.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude','Longitude','Pack']].values
    plt.subplot(25,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("Latitude, Longitude of Pack 9,2,1, and 10 given SnowDepth")
plt.tight_layout()  

plt.show()

In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack92110.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=105,n_clusters=4)
    t.fit(data)
    # Use subplots to display diagram and barcode side by side
    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    #gudhi.plot_persistence_barcode(persistence=data,legend=True,axes=axes[0])
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 9,2,1 and 10 given SnowDepth") 
    t.plot_diagram()
    print(t.diagram_)

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(1, figsize=(15, 25)) 
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack92110.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=105,n_clusters=4)
    t.fit(data)
    #t.plot_diagram()
    labels = [9 if i ==0 else 2 if i==1 else 1 if i==2 else 10 for i in t.labels_]
    plt.subplot(25,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 
plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 9,2,1 and 10 given SnowDepth")
plt.tight_layout()    
plt.show()

In [ ]:
#print(df_pack543["SnowDepth"])
fig = plt.figure(1, figsize=(15, 25)) 
for i,gp in df_pack543.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude','Longitude','Pack']].values
    plt.subplot(25,4,i)
    scatter=plt.scatter(X[:,0], X[:,1], c=X[:,2],cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("Latitude, Longitude of Pack 5,4, and 3 given SnowDepth")
plt.tight_layout()  

plt.show()

In [ ]:
import gudhi
from gudhi.clustering.tomato import Tomato
for (i,gp) in df_pack543.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']]
    data=X.values
    t = Tomato(graph_type='knn',density_type='logDTM', k=105,n_clusters=3)
    t.fit(data)
    # Use subplots to display diagram and barcode side by side
    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    #gudhi.plot_persistence_barcode(persistence=data,legend=True,axes=axes[0])
    plt.title("ToMATo Persistence Barcode of Lat./Long. in Pack 5,4 and 3 given SnowDepth") 
    t.plot_diagram()
    print(t.diagram_)

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(1, figsize=(15, 25)) 
#fig.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Pack 6 and 7 per Day")
import gudhi
from gudhi.clustering.tomato import Tomato

# fig = plt.figure(1, figsize=(10, 10)) 
for (i,gp) in df_pack543.groupby("SnowDepth"): 
#     display(i)
#     display(gp.head(2))
    X=gp[['Latitude', 'Longitude']].values
    #scaler = StandardScaler()
    #X = scaler.fit_transform(pd.DataFrame(X))
    data=X
    t = Tomato(graph_type='knn',density_type='logDTM', k=105,n_clusters=3)
    t.fit(data)
    #t.plot_diagram()
    labels = [5 if i ==0 else 4 if i==1 else 3 for i in t.labels_]
    plt.subplot(25,4,i)
    scatter=plt.scatter(data[:,0],data[:,1],marker='.',s=90,c=labels,cmap='coolwarm')
    #plt.legend(*scatter.legend_elements(),title="Pack")
    plt.legend(*scatter.legend_elements(),title="Pack",bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0.)
    plt.xlabel('Latitude'); plt.ylabel('Longitude'); 

plt.suptitle("ToMATo Persistence-based clustering Scatterplot of Latitude/Longitude of Packs 5, 4 and 3 given SnowDepth")   
plt.tight_layout()    
plt.show()

# tcount=np.bincount(t.labels_)
# print(tcount)
# t_bincount=(t.labels_)
# print(t_bincount)

# print("Number of tomato clusters for pack 5 is ",tcount[0])
# print("Number of tomato clusters for pack 4 is ", tcount[1])
# print("Number of tomato clusters for pack 3 is ", tcount[2])
#print("Number of tomato clusters for pack undecided is ", tcount[3])